In [ ]:
%cd ../..
%load_ext autoreload

%autoreload 2

In [ ]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from emv.utils import dataframe_from_hdf5
import emv.features.pose as pose
from emv.features.pose import process_video, draw_pose, get_frame, load_all_poses, process_all_poses
from emv.features.pose import format_keypoints_to_read, keypoint_name_to_id
from emv.features.pose import KEYPOINTS_NAMES, CONNECTIONS, ANGLES_ASSOCIATIONS
from emv.client.get_content import get_features

archive_path = "/mnt/g/ioc/data/"
os.path.isdir(archive_path)

sequences_folder = "/mnt/g/ioc/sequences/"
pose_folder = "data/test_poses/"

MODEL = pose.PifPafModel.fast

# Local extraction

In [ ]:
data = dataframe_from_hdf5(archive_path, "metadata")

In [ ]:
data.head()

In [ ]:
sequences_folder = "/mnt/g/ioc/sequences/"
extracted_sequences = os.listdir(sequences_folder)
data = data[data.guid.map(lambda x: x in extracted_sequences)]
data = data[data.duration_sec < 600]

In [ ]:
data.sport.value_counts()

In [ ]:
top_sports = data.sport.value_counts().index.tolist()
top_spoemv.remove("Non-Sport")

In [ ]:
data = data[data.sport.map(lambda x: x in top_sports)]

In [ ]:
N_SEQS_PER_SPORT = 100
sample = data.groupby("sport").sample(N_SEQS_PER_SPORT)
len(sample)

In [ ]:
sequences_videos = []
for root, dirs, files in os.walk(sequences_folder):
    for file in files:
        if file.endswith('.mp4'):
            sequences_videos.append(os.path.join(root, file))

print(len(sequences_videos))

In [ ]:
data["video_path"] = data.guid.map(lambda x: [f for f in sequences_videos if x in f][0])

In [ ]:
data.apply(lambda df: process_video(
    MODEL, 
    Path(f"{pose_folder}{df["guid"]}.jl"), 
    Path(df["video_path"]), 
    skip_frame=10
), axis = 1)

### Load extracted poses

In [ ]:
poses = load_all_poses(pose_folder)

# Load poses on S3

In [ ]:
poses = get_features(feature_type="pose", page_size=100, max_features=100000)
pose_df = process_all_poses(poses, merge_metadata = True)

In [ ]:
pose_df.sport.value_counts()

In [ ]:
pose_df.to_csv("data/pose_df.csv")

# Check results

In [ ]:
# create a grid of subplots
nrows = 2
ncols = 6
fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(ncols * 3, nrows * 3))
axs = axs.flatten()
# iterate over the poses and draw each pose in a subplot
sample_poses = pose_df.sample(nrows * ncols).reset_index(drop=True)
for i, pose in sample_poses.iterrows():
    draw_pose(pose, ax = axs[i], cut = True)

plt.show()

### Check angles

In [ ]:
def draw_angles(pose, angle_id, ax = None, cut=True, threshold=0.1):
    keypoints = pose["keypoints"]

    # Do not draw non-detected keypoints
    #connections = [c for c in connections if keypoints[KEYPOINTS_NAMES.index(c[0])][2] > threshold and keypoints[KEYPOINTS_NAMES.index(c[1])][2] > threshold]
    #keypoints = [k for k in keypoints if k[2] > threshold]
    
    frame = get_frame(pose["video_name"], pose["frame_number"])

    if ax is None:
        fig, ax = plt.subplots(figsize=(6,6))

    ax.imshow(frame)
    ax.scatter([k[0] for k in keypoints if k[2] > threshold], [k[1] for k in keypoints if k[2] > threshold], s=10)
    for c in CONNECTIONS:
        k1 = keypoints[KEYPOINTS_NAMES.index(c[0])]
        k2 = keypoints[KEYPOINTS_NAMES.index(c[1])]
        if k1[2] > threshold and k2[2] > threshold:
            ax.plot([k1[0], k2[0]], 
                    [k1[1], k2[1]], 
                    linewidth=0.5, color='black')
        
    # Draw input angle (k1,k2,k3) on the plot as lines between the keypoints, with text showing the angle value
    angle_value = pose["angle_vec"][angle_id]
    angle_name = list(ANGLES_ASSOCIATIONS.keys())[angle_id]
    angle_keypoints = ANGLES_ASSOCIATIONS[angle_name]
    k1 = keypoints[keypoint_name_to_id(angle_keypoints[0])]
    k2 = keypoints[keypoint_name_to_id(angle_keypoints[1])]
    k3 = keypoints[keypoint_name_to_id(angle_keypoints[2])]
    if k1[2] > threshold and k2[2] > threshold and k3[2] > threshold:
        if k1[2] > threshold and k2[2] > threshold and k3[2] > threshold:
            ax.plot([k1[0], k2[0]], 
                [k1[1], k2[1]], 
                linewidth=1, color='red')
            ax.plot([k3[0], k2[0]], 
                [k3[1], k2[1]], 
                linewidth=1, color='red')
        ax.text(k2[0], k2[1], f"({int(180 * angle_value)},{angle_value:.2f})", color="black", fontsize=12)


    # cut frame to bbox
    bbox = pose["bbox"]
    if cut:
        ax.set_xlim(int(bbox[0]),int(bbox[0] + bbox[2]))
        ax.set_ylim(int(bbox[1] + bbox[3]), int(bbox[1]))
        
    ax.axis("off")
    ax.set_aspect('equal')
    ax.set_title(angle_name)
    plt.tight_layout()

    return ax

In [ ]:
fig, axs = plt.subplots(nrows=3, ncols=3, figsize=(3 * 3, 3 * 3))
axs = axs.flatten()

for i in range(len(ANGLES_ASSOCIATIONS.keys())):
    draw_angles(poses[0], i, ax = axs[i], cut = True)
plt.show()